<a href="https://colab.research.google.com/github/zetavg/twlm/blob/main/Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TWLM Demo

由上方選單選擇「**執行階段**」>「**全部執行**」("Runtime" > "Run all") 來啟動。

過程中會自動下載需要的模型，初次執行到可以使用大概需要五到八分鐘。

在「**4. 啟動 UI**」的步驟會輸出一個網址，點擊該網址可以開啟 UI。

In [ ]:
#@title 1. 安裝
#@markdown 第一次執行時，這個步驟需要花費大約一分鐘。

!rm -rf llm_tuner
!git clone -b twlm-demo https://github.com/zetavg/LLaMA-LoRA-Tuner.git llm_tuner
!pip install -r llm_tuner/requirements.txt
!pip install bitsandbytes

In [ ]:
#@title 2. 一些設定
#@markdown 如果有 Colab Pro 可以用，GPU 選 V100 或 A100 的話，可以把 `Config.default_load_in_8bit` 改成 `False`，模型的速度會更快、輸出品質會更好。

from llm_tuner.llm_tuner.config import Config, process_config

Config.demo_mode = True

Config.default_load_in_8bit = True
Config.default_device_map = {'': 0}
# Config.default_device_map = 'auto'

Config.ui_show_starter_tooltips = False
Config.ui_inference_open_options_by_default = False
Config.ui_chat_reminder_message = "語言模型可能會產生不正確的資訊。因訓練成本與免費版 Colab 記憶體限制，模型目前對長串對話的表現不是很好，在多輪對話後可能會開始回應重複的內容。如果對回應不滿意，可以嘗試按下「Regenerate Response」。<br/>對話紀錄只會保存在瀏覽器中，關閉 Colab 時就會消失，若有需要請自行備份 (展開左下角「Raw Data」查看相關資訊)。下方「Examples」有一些訊息範本可以選擇。"
Config.ui_model_and_prompt_template_select_notice = "在 Colab 上使用非預設的模型時，需要花費大約五分鐘來下載模型 (初次使用時)，以及大約一分鐘載入模型 (每次切換)，請耐心等候。語言模型與對話均在"
Config.ui_features = ['chat', 'inference', 'tools']

Config.ui_title = "[Colab] TWLM Demo"
Config.ui_subtitle = ""
Config.ui_emoji = "🦙🎛️"

Config.default_generation_config = {
    'temperature': 1,
    'top_k': 40,
    'top_p': 0.2,
    'num_beams': 2,
    'repetition_penalty': 2.4,
    'max_new_tokens': 800,
}
Config.default_generation_stop_sequence = "### Human:"

In [ ]:
#@title 3. 初始化
#@markdown 第一次執行時，這個步驟需要花費大約五分鐘下載模型。之後，需要花費約一分半鐘來載入模型。<br /><br />如果因為用盡記憶體而失敗，可以試著再執行一次。

from llm_tuner.llm_tuner.initialization import initialize
initialize()

In [ ]:
#@title 4. 啟動 UI { display-mode: "form" }
#@markdown 執行此區塊會輸出類似 <br />`Running on public URL: https://xxxx.gradio.live` <br/>的文字，點擊其中 `https://xxxx.gradio.live` 的網址即可開啟 UI。
#@markdown （雖然直接在底下使用也行，但開新的視窗來用比較好用）


import gradio as gr
from llm_tuner.llm_tuner.ui import (
    main_page, get_page_title, get_css_styles
)

with gr.Blocks(title=get_page_title(), css=get_css_styles()) as demo:
    main_page()

demo.queue(concurrency_count=1).launch(share=True, debug=True, server_name="127.0.0.1")

In [ ]:
# import torch
# from transformers import AutoModelForCausalLM
# m = AutoModelForCausalLM.from_pretrained(
#     'zh-tw-llm-dv/tw-pythia-6.9b-chat-v0_2-s2',
#     torch_dtype=torch.float16,
#     load_in_8bit=True,
#     device_map={'': 0},
# )